In [11]:
import numpy as np
import cPickle
import nltk
from nltk.corpus import stopwords
from collections import Counter
from sklearn.metrics.pairwise import pairwise_distances
import itertools
import string
from nltk.tokenize import sent_tokenize
from nltk.stem.porter import *
#import pyemd
#loading word embeddings
#wv = KeyedVectors.load_word2vec_format("../w2v/w2vemb.bin", binary=True)

# Visualize functions

In [12]:
def write_file(n_clusters_, labels, corpus, mapping_dict, test_corpus):
    for indice_cluster in range(n_clusters_):
        # print indice_cluster
        idx_list = np.where(labels == indice_cluster)[0]
        f1 = open('cluster_%s/%dth_cluster.txt' % (question_class, indice_cluster), 'w+')
        for idx in idx_list:
            f1.write('%s %s\n' % (corpus[mapping_dict[idx] - 1], test_corpus[idx]))
    f1.close()


def write_ne(nes, question_class, mapping_dict):
    f1 = open('ne_%s.txt' % question_class, 'w+')
    for idx, doc in enumerate(nes):
        f1.write('%s %s\n' % (mapping_dict[idx], ' '.join(doc)))
    f1.close()    

def wmd_compute(x1, x2):
    return wv.wmdistance(x1, x2)

def analyze_cluster(n_clusters_, labels, corpus, mapping_dict, test_corpus, filename):
    num = 0
    other_content = []
    for indice_cluster in range(n_clusters_):
        idx_list = np.where(labels == indice_cluster)[0]
        if len(idx_list) > 4:
            f1 = open('%s_%s/%dth_cluster.txt' % (filename, question_class, indice_cluster), 'w+')
            for idx in idx_list:
                f1.write('%s %s\n' % (corpus[mapping_dict[idx] - 1], test_corpus[idx]))
            f1.close()
        if len(idx_list) < 4:
            num = num + len(idx_list)
            for idx in idx_list:
                other_content.append(corpus[mapping_dict[idx] - 1])
    f1 = open('%s_%s/other_cluster.txt' % (filename, question_class), 'w+')
    for doc in other_content:
        f1.write('%s\n' % doc)
    f1.close()
    print num

# Reading Raw text

In [13]:
question_class = 'q8'
fname = '..//data//%s.txt' % question_class
with open(fname) as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip() for x in content]
print 'length of content', len(content)
f.close()
print content

length of content 774
['there is communication gap between the engineer and a boy who makes the job card . if the engineer is telling something he should also listen to the customer what he is saying . boy was instructed to give in writing but still that boy gave excuses that computer is close etc . this will take 1 hour , whenever we visit there they say that computer is not working .', 'time which they take for service it should be less . they should increase service staff at dealership .', 'after taking vehicle for repair , they took 16 to 17 days which should happen in 1 to 2 days .', 'they are taking more time for water servicing and for billing so they have to reduce it .', 'they should do their work quickly .', 'once there was small issue in my vehicle then i visited the service center . i had urgent work so i wanted my vehicle quickly but they didn \\? t provide it . also they had took much time for such a small work .', 'whatever time , they are committing they should be give 

In [14]:
# split comment with multi-sentence into multi-comments
content_new = []
for comment in content:
    sents = sent_tokenize(comment)
    if len(sents) > 1:
        for i in range(len(sents)):
            content_new.append(sents[i])
    else:
        content_new.append(comment)
print 'length of new content', len(content_new)
# print content_new
    

length of new content 1978


In [15]:
pos_new = nltk.pos_tag(nltk.word_tokenize("it is 10 km far away from city it should be nearby city within 2 3 km . "))
print pos_new

[('it', 'PRP'), ('is', 'VBZ'), ('10', 'CD'), ('km', 'NN'), ('far', 'RB'), ('away', 'RB'), ('from', 'IN'), ('city', 'NN'), ('it', 'PRP'), ('should', 'MD'), ('be', 'VB'), ('nearby', 'JJ'), ('city', 'NN'), ('within', 'IN'), ('2', 'CD'), ('3', 'CD'), ('km', 'NN'), ('.', '.')]


In [16]:
stemmer = PorterStemmer()
print stemmer.stem("deliver")

deliv


In [17]:

def rule_q1(sen, ne):
    clean_ne = list(set(ne))
#     remove_words = ["improv", "custom", "servic", "peopl","person","facil","avail","good",\
#                     "center","centr","car", "dealership", "vehicl", "toyota", "problem","work", "much",\
#                    "thing", "possibl","need"]   #stemmed
    remove_words = ['custom','hour','minut','time','car', 'vehicl','servic', 'work', 'day','week','dealership',\
                   'center', 'toyota', 'appoint', 'pm', 'problem', 'period', 'part', 'morn','improv',\
                   'hr', 'peopl','person', 'lot', 'need','even', 'henc', 'manag','area', 'thing', 'till',\
                   'place', 'job', 'compani','today', 'call', 'clock', 'o\'clock', 'min','chang',\
                   'half']  #stemmed
    clean_ne = [word for word in clean_ne if word not in remove_words]
    
    save_words = ['commit', 'inform', 'delay', 'wash','wast','repair', 'fix', 'bill','mention','charg',\
                 'exact','extra', 'mechan','clean', 'spare','coordin', 'commun', 'far', 'return','deliv']  #stemmed
    clean_ne = clean_ne + [stemmer.stem(word) for word in sen.split() if stemmer.stem(word) in save_words]
    
    # rules to merge keywords:
    if 'delay' in clean_ne:
        clean_ne[clean_ne.index('delay')] = 'commit'
    if 'extra' in clean_ne:
        clean_ne[clean_ne.index('extra')] = 'commit'
        
    if 'mention' in clean_ne:
        clean_ne[clean_ne.index('mention')] = 'inform'
    if 'coordin' in clean_ne:
        clean_ne[clean_ne.index('coordin')] = 'inform'
    if 'commun' in clean_ne:
        clean_ne[clean_ne.index('commun')] = 'inform'
    if 'how much time' in sen:
        clean_ne.append('inform')
        
    if 'return' in clean_ne:
        clean_ne[clean_ne.index('return')] = 'deliveri'
    if 'deliv' in clean_ne:
        clean_ne[clean_ne.index('deliv')] = 'deliveri'
        
    if 'accessori' in clean_ne:
        clean_ne[clean_ne.index('accessori')] = 'spare'
        
    if 'advisor' in clean_ne:
        clean_ne[clean_ne.index('advisor')] = 'staff'
    if 'laor' in clean_ne:
        clean_ne[clean_ne.index('laor')] = 'staff'
    if 'manpow' in clean_ne:
        clean_ne[clean_ne.index('manpow')] = 'staff'
    if 'man power' in sen:
        clean_ne.append('staff')
    if 'serviceman' in clean_ne:
        clean_ne[clean_ne.index('serviceman')] = 'staff'
    if 'worker' in clean_ne:
        clean_ne[clean_ne.index('worker')] = 'staff'
        
    if 'km' in clean_ne:
        clean_ne[clean_ne.index('km')] = 'distanc'
    if 'far' in clean_ne:
        clean_ne[clean_ne.index('far')] = 'distanc' 
    
    clean_ne = list(set(clean_ne))
    return clean_ne

def clean_corpus(x1):
    x_new = []
    for x in x1:
        if 'no improvement' in x:
            continue
        else:
            x_new.append(x)
    return x_new

def process_corpus(x1, pos_tags, general_stop):
    # split three categories: 1 no improvemnt 2 with noun 3 others
    doc_noimprove = []
    doc_nn = []
    nn_extracted = []
    doc_other = []
    for x in x1:
        if 'no improvement' in x:
            doc_noimprove.append(x)
        else:
                nn_list = []
                sen = x
                pos_new = nltk.pos_tag(nltk.word_tokenize(sen))
                for token in pos_new:
                    if token[1] in pos_tags and not token[0] in general_stop:
                        nn_list.append(token[0])
                #stemming
                for counter,word in enumerate(nn_list):
                    nn_list[counter] = stemmer.stem(word)
                # apply rule
                nn_list = rule_q1(sen, nn_list)
                    
                if nn_list != []:
                    nn_extracted.append(nn_list)
                    doc_nn.append(sen)
                else:
                    doc_other.append(sen)
    return doc_noimprove, [doc_nn, nn_extracted], doc_other
        
nn_corpus = []

# English stop words lists
stop_words = stopwords.words('english')
punctuation_list = [unicode(i) for i in string.punctuation]
for punctuation in punctuation_list:
    stop_words.append(punctuation)

pos_tags = ['NN', 'NNS']

doc1, doc2, doc3 = process_corpus(content_new, pos_tags, stop_words)

doc_nn, nn_extracted = doc2[0], doc2[1]
print 'signle sentence with nn', len(doc_nn)
print 'no comments', len(doc1)
print 'single sentence without nn', len(doc3)
print nn_extracted

signle sentence with nn 1106
no comments 8
single sentence without nn 864
[['boy', u'engin', u'commun', 'gap', 'inform', 'card'], [u'someth', u'engin'], [u'write', 'boy', u'excus', u'comput', 'etc'], [u'comput'], ['staff'], ['repair'], ['water', u'bill'], [u'issu'], [u'commit'], [u'commit', 'approx', u'timelin'], ['distanc'], ['inform', 'staff'], [u'statu'], ['technician', 'inform', 'lack', 'relationship', u'commun'], ['deliveri'], [u'branch', u'velacheri'], [u'commit', 'wait'], [u'decreas'], ['deliveri'], [u'everyth'], ['queue', 'visit'], ['paper', u'assist'], ['wash'], [u'capac', 'number'], [u'record'], [u'capac', 'staff'], ['inform'], [u'avail', 'spare'], [u'deliveri', 'commit'], [u'deliveri', 'commit'], [u'deliveri'], [u'batteri', 'deliveri', 'distanc'], [u'issu', u'condit', 'side', 'commit'], ['side'], ['afternoon'], ['commit', 'request'], ['spare', 'dealer', 'wait'], ['wheel', 'town', u'facil', u'allot'], ['third'], ['power'], ['staff'], [u'commit'], [u'mechan', 'staff'], [u'char

In [18]:
from collections import Counter
def df_count(x1):
    text = []
    for ab in x1:
        text = text + ab
    df = Counter(text)
    return df
df = df_count(nn_extracted)
print df

Counter({'deliveri': 239, u'commit': 212, 'staff': 129, 'inform': 73, u'wast': 29, 'wash': 25, 'distanc': 25, 'repair': 23, u'fix': 18, u'deliv': 18, u'bill': 18, u'charg': 16, 'exact': 15, u'issu': 13, u'mechan': 12, 'wait': 11, 'spare': 11, 'oil': 11, 'card': 9, u'facil': 9, 'clean': 9, u'increas': 9, u'process': 8, 'return': 8, 'care': 8, 'tomorrow': 8, 'hand': 8, u'attent': 7, 'delay': 7, 'labor': 7, 'number': 7, 'space': 7, 'showroom': 7, 'power': 7, 'paper': 6, 'lunch': 6, u'advanc': 6, 'reason': 6, u'centr': 6, 'speed': 6, 'complaint': 6, 'fast': 6, 'way': 6, u'depart': 5, 'date': 5, 'side': 5, u'qualiti': 5, u'comput': 5, u'plan': 5, u'book': 5, 'visit': 5, 'wise': 5, 'month': 4, 'water': 4, u'valu': 4, u'commun': 4, u'purpos': 4, 'man': 4, u'sm': 4, 'feedback': 4, 'name': 4, u'anyth': 4, u'capac': 4, 'owner': 4, u'citi': 4, 'system': 4, u'noth': 4, u'insur': 4, u'express': 4, 'frame': 4, u'respons': 4, u'someth': 4, 'ramp': 4, u'suggest': 4, u'everyth': 4, 'home': 4, u'machin'

# use one suitable aspect to represent each sentence

In [19]:
import heapq
import os
def filter_rule_q1(doc, xth, test_corpus, original_corpus = doc_nn):
    apply_rule = False
    # print original_corpus[xth]
    # rules (priority from high to low)
         
    return apply_rule
    
def filter_ne(test_corpus, df):  # assuming each review contain one aspect
    for xth, doc in enumerate(test_corpus):
        if len(doc)>1:
            apply_rule = filter_rule_q1(doc, xth, test_corpus)
            if not apply_rule:
                df_words = [df[word] for word in doc]
                idx =  heapq.nlargest(1, xrange(len(df_words)), key=df_words.__getitem__)
                test_corpus[xth] = [doc[ith] for ith in idx]
    return test_corpus
nn_clean = filter_ne(nn_extracted, df)
df = df_count(nn_clean)
dict_map = dict(df.most_common())
print df



Counter({'deliveri': 239, u'commit': 159, 'staff': 117, 'inform': 49, u'wast': 27, 'distanc': 22, 'repair': 21, 'wash': 21, u'charg': 14, 'fix': 13, u'issu': 12, u'bill': 11, 'spare': 10, u'mechan': 8, 'exact': 8, u'facil': 8, 'oil': 8, 'wait': 7, 'clean': 6, 'number': 6, 'paper': 5, 'lunch': 5, 'care': 5, u'qualiti': 5, u'process': 5, 'complaint': 4, 'speed': 4, u'advanc': 4, 'reason': 4, 'card': 4, 'tomorrow': 4, 'fast': 4, 'wise': 4, u'comput': 4, u'centr': 4, u'suggest': 4, u'book': 4, u'noth': 4, 'labor': 3, 'name': 3, u'valu': 3, u'respons': 3, 'space': 3, u'someth': 3, 'showroom': 3, 'power': 3, 'screen': 3, 'ramp': 3, 'visit': 3, 'start': 3, u'purpos': 3, 'counter': 3, 'year': 3, 'dealer': 3, u'insur': 3, 'shop': 2, 'feedback': 2, 'money': 2, u'month': 2, 'fine': 2, u'busi': 2, 'rs': 2, u'middl': 2, u'other': 2, u'pass': 2, u'express': 2, u'anyth': 2, 'side': 2, 'home': 2, 'water': 2, 'c': 2, 'ok': 2, u'outsid': 2, u'messag': 2, 'owner': 2, u'prioriti': 2, u'christma': 2, u'cit

# Decide main cluster and write to file

In [23]:
def write_file(corpus, idx_list, word):
    if not os.path.isdir("cluster/%s" %word):
            os.mkdir("cluster/%s" %word)
    f1 = open('cluster/%s/%s_comment.txt' % (word, word), 'w+')
    for idx in idx_list:
        f1.write('%s\n' %corpus[idx])
    f1.close()
    
    
def main_category(df_list, nn_clean, corpus):
    if (not os.path.isdir("cluster")):
        os.mkdir("cluster")
    name_list = {}
    clustered_index = []
    major_list = [word for word in df_list if df_list[word]>5]
    print "majot list:\n", major_list
    for word in major_list:
        idx_set = []
        for idx, doc in enumerate(nn_clean):
            if word in doc:
                idx_set.append(idx)
        write_file(doc_nn, idx_set, word)
        name_list[word] = idx_set
        clustered_index = clustered_index + idx_set
    unclustered_index = [x for x in range(len(nn_clean)) if x not in clustered_index]
    print "num of clustered:",len(clustered_index)
    print "num of others:", len(unclustered_index)
    name_list['others'] = unclustered_index
    write_file(doc_nn, unclustered_index, 'others')
    return name_list

name_list = main_category(dict_map, nn_clean,doc_nn)
# print name_list['call']

majot list:
['distanc', 'staff', 'fix', 'number', u'mechan', 'exact', 'spare', u'commit', 'repair', u'wast', 'wash', u'facil', 'wait', 'deliveri', u'bill', 'oil', u'charg', u'issu', 'inform', 'clean']
num of clustered: 766
num of others: 340


In [ ]:
clustered_index = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 17, 18, 19, 20, 22, \
                   23, 24, 25, 26, 27, 28, 29, 30, 32, 33, 34, 36, 37, 38, 39]
clustered_index = set(clustered_index)
print clustered_index
print set(range(40))
print set(range(40))-clustered_index